# Capturing neutral face video

In [ ]:
import numpy as np
import pickle
from matplotlib import pyplot as plt

import face_alignment
import time
import cv2
import scipy.optimize as opt
from calibration.undistort import undistort

def draw_marks(image, marks, color=(0, 255, 0)):
    """
    Draw the facial landmarks on an image
    Parameters
    ----------
    image : np.uint8
        Image on which landmarks are to be drawn.
    marks : list or numpy array
        Facial landmark points
    color : tuple, optional
        Color to which landmarks are to be drawn with. The default is (0, 255, 0).
    Returns
    -------
    None.
    """
    for mark in marks:
        cv2.circle(image, (mark[0], mark[1]), 2, color, -1, cv2.LINE_AA)
        

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cuda')

# this function compares the differences between two sets of points
# each column is a point
def err(p1, p2):
    # hopefully this is actually correct
    return sum(sum((p1 - p2)*(p1 - p2)))



# x is an array of 9 elements
def to_optimize(x, original_points, target_points):
    # reshape it to be a 3x3 transformation matrix
    transform = x.reshape((3,3))
    temp = transform@original_points
    result = np.zeros((2,original_points.shape[1]))
    for i in range(original_points.shape[1]):
        result[0,i] = temp[0,i]/temp[2,i]
        result[1,i] = temp[1,i]/temp[2,i]
    return err(result, target_points[:2,:])


def crop_image(image):
    return image[:,80:-80,:]

def capture_initial_video():
    original_points = 0
    target_points = 0
    cap = cv2.VideoCapture(0)
    
    ## capture jawline with headset on and save the feature pts for alignment 
    while True:
        res, frame = cap.read()
        if not res:
            print('no frame')
            continue
        cv2.imshow(winname="RAW FRAME", mat=frame)

        frame = crop_image(frame)
        frame = cv2.resize(frame,(256,256))
        
        preds = fa.get_landmarks(frame)
        if(preds):
            headset_features = np.concatenate((preds[0].astype(int)[:17],preds[0].astype(int)[49:61]))
            draw_marks(frame,headset_features,color = (255,0,0))
        cv2.imshow(winname="Face", mat=cv2.resize(frame,(720,720)))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            target_points = np.concatenate((preds[0].astype(int)[:17],preds[0].astype(int)[49:61]))
            target_points = np.append(target_points, np.ones((17,1)), axis=1)
            target_points = target_points.T
            print("got target points with shape", target_points.shape)
            break   
        
    video_out = cv2.VideoWriter('./init_video.mp4',cv2.VideoWriter_fourcc(*'XVID'),25,(256,256))
    print('now trying to find a good no-headset match')
    record = False
    
    ## align 
    result_marks = 0
    generated_result = False
    while True:
        res, frame = cap.read()
        if not res:
            print('no frame')
            continue

        frame = crop_image(frame)
        frame = cv2.resize(frame,(256,256))
        # show the image
        preds = fa.get_landmarks(frame)
        if(preds):
            new_face = preds[0].astype(int)
            clean_frame = frame.copy()

            draw_marks(frame,new_face,color = (0,0,255))
        if generated_result:
            print("generating result")
            draw_marks(frame,result_marks, color = (0,255,0))
        draw_marks(frame,headset_features,color = (255,0,0))
        cv2.imshow(winname="Face", mat=cv2.resize(frame,(720,720)))

        pressed_key = cv2.waitKey(5) & 0xFF

        if(pressed_key == ord('q')):
            print("stop recording")
            break
        elif(pressed_key == ord('r')):
            print('started recording')
            record = True
            original_points = np.concatenate((preds[0].astype(int)[:17],preds[0].astype(int)[49:61]))
            original_points = np.append(original_points, np.ones((original_points.shape[0],1)), axis=1)
            original_points = original_points.T
            
            result_marks = original_points
            
            generated_result = True

        if(record):
            video_out.write(clean_frame)
    # When everything done, release the video capture and video write objects
    cap.release()
    video_out.release()
    # Close all windows
    cv2.destroyAllWindows()
    pickle.dump(clean_frame,open('./starting_picture.p','wb'))
    return clean_frame

print("run")

capture_initial_video()
cv2.destroyAllWindows()

In [ ]:
import cv2
## Take the first frame of the video w/ headset on 
cap1 = cv2.VideoCapture('./init_video.mp4')
for i in range(30):
    res, frame = cap1.read()
    cv2.imshow('aaaa',frame)
    cv2.waitKey(100)
cv2.imwrite('./init_img2.png',frame)
cv2.destroyAllWindows()

In [ ]:
#### import matplotlib
# matplotlib.use('Agg')
import os, sys
import yaml
from argparse import ArgumentParser
from tqdm import tqdm

import imageio
import numpy as np
from skimage.transform import resize
from skimage import img_as_ubyte
import torch
from sync_batchnorm import DataParallelWithCallback

from modules.generator import OcclusionAwareGenerator
from modules.keypoint_detector import KPDetector
from animate import normalize_kp
from scipy.spatial import ConvexHull
import pdb
#import pyvirtualcam
import time
from calibration.undistort import undistort
import cv2
import copy

USE_RECORDED_VIDEO = True
MERGE_HEIGHT = 128
def crop_image(image):
    return image[:,80:-80,:]

if sys.version_info[0] < 3:
    raise Exception("You must use Python 3 or higher. Recommended version is Python 3.7")

def draw_marks(image, marks, color=(0, 255, 0)):
    """
    Draw the facial landmarks on an image
    Parameters
    ----------
    image : np.uint8
        Image on which landmarks are to be drawn.
    marks : list or numpy array
        Facial landmark points
    color : tuple, optional
        Color to which landmarks are to be drawn with. The default is (0, 255, 0).
    Returns
    -------
    None.
    """
    for mark in marks:
        cv2.circle(image, (mark[0], mark[1]), 2, color, -1, cv2.LINE_AA)    
    
def load_checkpoints(config_path, checkpoint_path, cpu=False):

    with open(config_path) as f:
        config = yaml.safe_load(f)

    generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
    if not cpu:
        generator.cuda()

    kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                             **config['model_params']['common_params'])
    if not cpu:
        kp_detector.cuda()
    
    if cpu:
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    else:
        checkpoint = torch.load(checkpoint_path)
 
    generator.load_state_dict(checkpoint['generator'])
    kp_detector.load_state_dict(checkpoint['kp_detector'])
    
    if not cpu:
        generator = DataParallelWithCallback(generator)
        kp_detector = DataParallelWithCallback(kp_detector)

    generator.eval()
    kp_detector.eval()
    
    return generator, kp_detector

## make animation based on a video (sequence of frames), essentially the same as create_frame but for videos
def make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True, cpu=False):
    with torch.no_grad():
        predictions = []
        source = torch.tensor(source_image[np.newaxis].astype(np.float32)).permute(0, 3, 1, 2)
        if not cpu:
            source = source.cuda()
        driving = torch.tensor(np.array(driving_video)[np.newaxis].astype(np.float32)).permute(0, 4, 1, 2, 3)
        kp_source = kp_detector(source)
        kp_driving_initial = kp_detector(driving[:, :, 0])

        for frame_idx in tqdm(range(driving.shape[2])):
            driving_frame = driving[:, :, frame_idx]
            if not cpu:
                driving_frame = driving_frame.cuda()
            kp_driving = kp_detector(driving_frame)
            
            kp_norm = normalize_kp(kp_source=kp_source, kp_driving=kp_driving,
                                   kp_driving_initial=kp_driving_initial, use_relative_movement=relative,
                                   use_relative_jacobian=relative, adapt_movement_scale=adapt_movement_scale)
            out = generator(source, kp_source=kp_source, kp_driving=kp_norm)
            predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])
    return predictions


def create_frame(driving_video, generator, kp_detector,kp_source,kp_driving_initial,\
                 source, relative=True, adapt_movement_scale=True, cpu=False,):
    with torch.no_grad():
        if not cpu:
            source = source.cuda()
        driving_frame = torch.tensor(driving_video[np.newaxis].astype(np.float32)).permute(0, 3, 1, 2)
        if not cpu:
            driving_frame = driving_frame.cuda()
        ## detect driving frame kp
        kp_driving = kp_detector(driving_frame)
        kp_norm = normalize_kp(kp_source=kp_source, kp_driving=kp_driving,
                               kp_driving_initial=kp_driving_initial, use_relative_movement=relative,
                               use_relative_jacobian=relative, adapt_movement_scale=adapt_movement_scale)
        out = generator(source, kp_source=kp_source, kp_driving=kp_norm)
        return np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0]



source_image = (imageio.imread('./init_img2.png')).astype(np.uint8)
reader2 = cv2.VideoCapture('./init_video.mp4')
driving_video = []
regular_video = []

def pre_process_frame(frame, crop = True, cvt_color = True):
    #frame = frame[:,80:-80,:]
    if crop:
        frame = crop_image(frame)
    frame = cv2.resize(frame,(256,256))
    if cvt_color:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
    frame = [resize(img, (256, 256))[..., :3] for img in [frame]]
    return frame
        
while(1):
    ret,frame = reader2.read()
    if(ret):
        regular_video.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    else:
        break

        
        
## first image of the natural blinking video
source_image = resize(source_image, (256, 256))[..., :3]
## resize the natural blinking video 
regular_video = [resize(frame, (256, 256))[..., :3] for frame in regular_video]

if not USE_RECORDED_VIDEO:
    ## make sure camera is working 
    cap = cv2.VideoCapture(0)
    res, frame = cap.read()
    while(res is not True):
        res, frame = cap.read()
        print('retry')
        time.sleep(0.3)
    driving_video = pre_process_frame(frame)
else:
    saved_video_reader = cv2.VideoCapture('./video_headset_on.mp4')
    saved_video = []
    while(1):
        ret,frame = saved_video_reader.read()
        if(ret):
            saved_video.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        else:
            break
    driving_video = pre_process_frame(saved_video[0], crop =False, cvt_color = False)

## Add the neutral blinking video to the top part 
driving_video[0][:MERGE_HEIGHT,:] = regular_video[0][:MERGE_HEIGHT,:]
# cv2.imshow(winname="Face", mat=cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR))

## Load the pre-trained first-order model
generator, kp_detector = load_checkpoints(config_path='./config/vox-adv-256.yaml', \
                                          checkpoint_path='./vox-adv-cpk.pth.tar', cpu=False)

## torchlize first frame of neutral face
source = torch.tensor(source_image[np.newaxis].astype(np.float32)).permute(0, 3, 1, 2)
source = source.cuda()
## the kp of this frame, which remains to be same throughout online streaming 
kp_source = kp_detector(source)

## Torchlize first frame of driving/online video 
driving = torch.tensor(np.array(driving_video)[np.newaxis].astype(np.float32)).permute(0, 4, 1, 2, 3)
relative = True
adapt_movement_scale = True

predictions = []
kp_driving_initial = kp_detector(driving[:, :, 0])


## Stitching-based 
height_black_band = [105, 120]
lateral_shift = 2
left_right_black_size = 90
bottom_black_size = 70



counter = 0

#with pyvirtualcam.Camera(width=256, height=256, fps=30,device='/dev/video5') as cam:  
#while(1):
frame_to_save_indeces = [0,10,20,30,40,60,70,80,90, 25, 50, 75, 100, 125, 150, 175, 250]
while counter < 251: #len(saved_video):
    start = time.time()
    if not USE_RECORDED_VIDEO:
        res, frame = cap.read()
        if not res:
            print('no frame')
            continue
        cv2.imwrite("normalframe.jpg", frame);
        frame = pre_process_frame(frame)
    else:
        frame = saved_video[counter%len(saved_video)]
        frame = pre_process_frame(frame, crop = False, cvt_color = False)
    
    ## stitching-based
    stitch_frame = copy.copy(frame[0])
    stitch_frame[:height_black_band[0],:] = regular_video[counter%len(regular_video)][:height_black_band[0],:] #256 x 256

    stitch_frame[height_black_band[0]:height_black_band[1],:] = np.zeros((height_black_band[1] - height_black_band[0], 256, 3))

    ## shift chin 
    stitch_frame[height_black_band[1]:,left_right_black_size + lateral_shift:256 - left_right_black_size + lateral_shift,:] = \
        stitch_frame[height_black_band[1]:,left_right_black_size :256 - left_right_black_size,:]

    stitch_frame[height_black_band[1]:256,0:left_right_black_size] = \
        np.zeros((256 - height_black_band[1], left_right_black_size, 3))
    stitch_frame[height_black_band[1]:256,-left_right_black_size:] = \
        np.zeros((256 - height_black_band[1], left_right_black_size, 3))
    stitch_frame[-bottom_black_size:,:] = np.zeros((bottom_black_size, 256,3))
    if counter in frame_to_save_indeces:
        tmp = cv2.resize(stitch_frame,(720,720))
        tmp[:,:,[0,2]] = tmp[:,:,[2,0]]
        cv2.imwrite(f'black_frame_{counter}.jpg', (tmp*255).astype(np.uint8))
    
    
    
    raw_frame = copy.copy(frame[0])
    tmp = cv2.resize(raw_frame,(720,720))
    tmp[:,:,[0,2]] = tmp[:,:,[2,0]]
    if counter in frame_to_save_indeces:
        cv2.imwrite(f'raw_frame_{counter}.jpg', (tmp*255).astype(np.uint8))
        
    ## Add the neutral blinking video to the top part 
    frame[0][:MERGE_HEIGHT,:] = regular_video[counter%len(regular_video)][:MERGE_HEIGHT,:]   
    
    output = create_frame(frame[0],generator,kp_detector,kp_source,kp_driving_initial,source,relative=True, adapt_movement_scale=True, cpu=False)
    output255 = (output*255).astype(np.uint8)
    
    output255 = cv2.resize(output255, (720,720))
    output255[:,:,[0,2]] = output255[:,:,[2,0]]
    if counter in frame_to_save_indeces:
        cv2.imwrite(f'learning_output_{counter}.jpg', output255)
    #cv2.imshow('aaaaa',output255)
    #cv2.waitKey(1)

    ## debugging visualization
    single_frame = frame[0]
    #draw_marks(single_frame, kp_driving[0].astype(int))
    tmp = cv2.resize(single_frame,(720,720))
    tmp[:,:,[0,2]] = tmp[:,:,[2,0]] 
    #cv2.imshow("split frame",tmp)

    if counter in frame_to_save_indeces:
        cv2.imwrite(f'stitched_frame_{counter}.jpg', (tmp*255).astype(np.uint8))
        
        
    counter = (counter+1)%len(regular_video)

